In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from collections import OrderedDict
from random import randint
import random
import decimal
from operator import add
from operator import itemgetter

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_hidden_layers = 9
max_neurons_per_layer = 100
min_learning_rate = .0001
max_learning_rate = .01
acti_func = ["softmax", "elu", "selu", "softplus", "softsign", 
             "relu", "tanh", "sigmoid", "hard_sigmoid", "linear"]
num_individuals = 16
loss_weight = 10
accuracy_weight = 10
time_weight = 10

In [3]:
def load_data():
    iris_data = load_iris()
    print('Example Data:')
    print(iris_data.data[:5])
    print('Example Labels:')
    print(iris_data.target[:5])
    global iris_data

<ipython-input-3-11c664384486>:7: SyntaxWarning: name 'iris_data' is assigned to before global declaration
  global iris_data


In [4]:
def process_data():
    x = iris_data.data
    y_ = iris_data.target.reshape(-1,1)

    encoder = OneHotEncoder(sparse=False)
    y = encoder.fit_transform(y_)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.20)
    global train_x, test_x, train_y, test_y

<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y


In [6]:
def create_network(model_num):
    model_name = "Model"+str(model_num)
    model_desc = {'name':model_name}
    print(model_name)
    model_name = Sequential()
    num_hidden_layers = randint(0, max_hidden_layers)
    i = 0
    model_name.add(Dense(10, input_shape=(4,), activation='relu', name='input'))
    while i < num_hidden_layers:
        name = 'hidden'+str(i)
        num_neurons = randint(1, max_neurons_per_layer)
        activation = acti_func[randint(0, len(acti_func)-1)]
        model_desc.update({'hidden'+str(i) : (num_neurons, activation)})
        #print(activation)
        model_name.add(Dense(num_neurons, activation=activation, name=name))
        i = i+1
    model_name.add(Dense(3, activation='softmax', name='output'))
    global model_name
    global model_desc

<ipython-input-6-0e67b9f64f4e>:18: SyntaxWarning: name 'model_name' is assigned to before global declaration
  global model_name
<ipython-input-6-0e67b9f64f4e>:19: SyntaxWarning: name 'model_desc' is assigned to before global declaration
  global model_desc


In [7]:
def lr_and_optimizer(model_name):
    learning_rate = random.uniform(min_learning_rate, max_learning_rate)
    #print(learning_rate)
    model_desc.update({'learning_rate' : learning_rate})
    optimizer = Adam(lr=learning_rate)
    model_desc.update({'optimizer' : 'Adam'})
    model_name.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
def print_summary(model_name):
    print('Neural Net Model Summary: ')
    print(model_name.summary())

In [9]:
def train_and_test(model_name):
    model_name.fit(train_x, train_y, verbose = 2, batch_size = 5, epochs = 200)
    results = model_name.evaluate(test_x, test_y)
    global results

<ipython-input-9-915b75c55b69>:4: SyntaxWarning: name 'results' is assigned to before global declaration
  global results


In [10]:
def individual(model_num):
    create_network(model_num)
    #print_summary(model_name)
    lr_and_optimizer(model_name)
    train_and_test(model_name)
    loss_fitness =  (1/float(results[0]))*loss_weight
    acc_fitness = float(results[1])*accuracy_weight
    total_fitness = loss_fitness + acc_fitness
    model_desc.update({'loss_fitness':loss_fitness})
    model_desc.update({'accuracy_fitness':acc_fitness})
    model_desc.update({'total_fitness':total_fitness})
    print(model_name)
    print(model_desc)

In [11]:
def population():
    population_desc = []
    for x in xrange(num_individuals):
        individual(model_num = x)
        population_desc.append(model_desc)
    print(population_desc)
    global population_desc

<ipython-input-11-662326261e74>:7: SyntaxWarning: name 'population_desc' is assigned to before global declaration
  global population_desc


In [13]:
def grade():    
    fitness_sum = 0
    'Find average fitness for a population.'
    for x in population_desc:
        fitness_sum = x.get("total_fitness") + fitness_sum
    avg_fitness = fitness_sum/len(population_desc)
    print(avg_fitness)
    global avg_fitness

<ipython-input-13-e0a3493f5de1>:8: SyntaxWarning: name 'avg_fitness' is assigned to before global declaration
  global avg_fitness


In [15]:
def sort_parents():
    parents = []
    for x in population_desc:
        if x.get("total_fitness") >= avg_fitness:
            parents.append((x.get('name'), x.get('total_fitness')))
            difference = x.get('total_fitness') - avg_fitness
            #print(x.get("name")+"    "+str(x.get("total_fitness"))+"    "+str(difference))
    def get_key(item):
        return item[1]
    parents_sorted = sorted(parents, key = get_key, reverse = True)

    if len(parents) % 2 !=0:
        parents_sorted.pop()
        print(parents_sorted)
    else:
        print(parents_sorted)

[('Model0', 3248.4254783613096), ('Model6', 2894.5059186549706), ('Model10', 1763.2754439147611), ('Model2', 1486.2712411098234)]


In [27]:
def create_child(child_num, model_1, model_2):
    parent1 = (item for item in population_desc if item["name"] == model_1).next()
    parent2 = (item for item in population_desc if item["name"] == model_2).next()
    print(parent1)
    print(parent2)
    new_learning_rate = (parent1.get("learning_rate")+parent2.get("learning_rate"))/2
    print(new_learning_rate)
    def parent_select():
        num = randint(0,1)
        if num == 0:
            return parent1
        else:
            return parent2
    model_name = "Child"+str(child_num)
    model_desc = {'name':child_name}
    print(child_name)
    model_name = Sequential()
    model_name.add(Dense(10, input_shape=(4,), activation='relu', name='input'))
    avg_hidden = 0
    parent1_hidden = 0
    parent2_hidden = 0
    for key in parent1.keys():
        if "hidden" in key:
            parent1_hidden = parent1_hidden + 1
    for key in parent2.keys():
        if "hidden" in key:
            parent2_hidden = parent2_hidden + 1
    avg_hidden = (parent1_hidden + parent2_hidden)/2
    print(avg_hidden)
    i = 0
    while i < avg_hidden:
        name = 'hidden'+str(i)
        try:
            num_neurons = parent_select().get(name)[0]
            activation = parent_select().get(name)[1]
        except:
            try:
                num_neurons = parent1.get(name)[0]
                activation = parent1.get(name)[1]
            except:
                num_neurons = parent2.get(name)[0]
                activation = parent2.get(name)[1]
        model_desc.update({'hidden'+str(i) : (num_neurons, activation)})
        #print(activation)
        model_name.add(Dense(num_neurons, activation=activation, name=name))
        i = i+1
    model_name.add(Dense(3, activation='softmax', name='output'))
    model_desc.update({'learning_rate' : new_learning_rate})
    optimizer = Adam(lr=new_learning_rate)
    model_desc.update({'optimizer' : 'Adam'})
    model_name.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
def child(child_num, model_1, model_2):
    create_child(child_num, model_1, model_2)
    print_summary(child_name)
    lr_and_optimizer(child_name)
    train_and_test(child_name)
    loss_fitness =  (1/float(results[0]))*loss_weight
    acc_fitness = float(results[1])*accuracy_weight
    total_fitness = loss_fitness + acc_fitness
    model_desc.update({'loss_fitness':loss_fitness})
    model_desc.update({'accuracy_fitness':acc_fitness})
    model_desc.update({'total_fitness':total_fitness})
    print(model_name)
    print(model_desc)

In [38]:
def child_populations():
    population_desc = []
    for x in xrange(len(parents)):
        try:
            child(x, parents_sorted[x*2][0], parents_sorted[(x*2)+1][0])
            population_desc.append(child_desc)
        except:
            pass
    print(population_desc)

In [39]:
child_populations()

{'hidden3': (83, 'elu'), 'hidden2': (33, 'softplus'), 'hidden1': (86, 'elu'), 'hidden0': (88, 'softsign'), 'optimizer': 'Adam', 'name': 'Model0', 'learning_rate': 0.0012489600449276383, 'loss_fitness': 3238.4254783613096, 'total_fitness': 3248.4254783613096, 'accuracy_fitness': 10.0}
{'hidden3': (87, 'softmax'), 'hidden2': (42, 'softplus'), 'hidden1': (16, 'sigmoid'), 'hidden0': (86, 'linear'), 'hidden7': (70, 'relu'), 'name': 'Model6', 'hidden5': (28, 'relu'), 'hidden4': (71, 'softplus'), 'learning_rate': 0.00013985712252201138, 'hidden8': (46, 'relu'), 'loss_fitness': 2884.5059186549706, 'accuracy_fitness': 10.0, 'total_fitness': 2894.5059186549706, 'optimizer': 'Adam', 'hidden6': (96, 'tanh')}
0.000694408583725
6
Neural Net Model Summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 10)                50        
_________________________________________________

Epoch 134/200
 - 0s - loss: 0.1065 - acc: 0.9583
Epoch 135/200
 - 0s - loss: 0.0726 - acc: 0.9667
Epoch 136/200
 - 0s - loss: 0.0690 - acc: 0.9833
Epoch 137/200
 - 0s - loss: 0.0645 - acc: 0.9833
Epoch 138/200
 - 0s - loss: 0.0788 - acc: 0.9667
Epoch 139/200
 - 0s - loss: 0.1177 - acc: 0.9500
Epoch 140/200
 - 0s - loss: 0.0561 - acc: 0.9583
Epoch 141/200
 - 0s - loss: 0.0692 - acc: 0.9750
Epoch 142/200
 - 0s - loss: 0.0580 - acc: 0.9833
Epoch 143/200
 - 0s - loss: 0.0622 - acc: 0.9833
Epoch 144/200
 - 0s - loss: 0.0605 - acc: 0.9833
Epoch 145/200
 - 0s - loss: 0.0866 - acc: 0.9667
Epoch 146/200
 - 0s - loss: 0.0650 - acc: 0.9833
Epoch 147/200
 - 0s - loss: 0.0881 - acc: 0.9667
Epoch 148/200
 - 0s - loss: 0.4367 - acc: 0.8000
Epoch 149/200
 - 0s - loss: 0.1679 - acc: 0.9333
Epoch 150/200
 - 0s - loss: 0.1552 - acc: 0.9417
Epoch 151/200
 - 0s - loss: 0.1488 - acc: 0.9417
Epoch 152/200
 - 0s - loss: 0.1517 - acc: 0.9250
Epoch 153/200
 - 0s - loss: 0.0704 - acc: 0.9667
Epoch 154/200
 - 0s 

 - 0s - loss: 0.0543 - acc: 0.9833
Epoch 61/200
 - 0s - loss: 0.0596 - acc: 0.9750
Epoch 62/200
 - 0s - loss: 0.0551 - acc: 0.9750
Epoch 63/200
 - 0s - loss: 0.0508 - acc: 0.9833
Epoch 64/200
 - 0s - loss: 0.0506 - acc: 0.9750
Epoch 65/200
 - 0s - loss: 0.0617 - acc: 0.9833
Epoch 66/200
 - 0s - loss: 0.0607 - acc: 0.9750
Epoch 67/200
 - 0s - loss: 0.0543 - acc: 0.9750
Epoch 68/200
 - 0s - loss: 0.0530 - acc: 0.9750
Epoch 69/200
 - 0s - loss: 0.0543 - acc: 0.9750
Epoch 70/200
 - 0s - loss: 0.0519 - acc: 0.9750
Epoch 71/200
 - 0s - loss: 0.0525 - acc: 0.9833
Epoch 72/200
 - 0s - loss: 0.0523 - acc: 0.9750
Epoch 73/200
 - 0s - loss: 0.0523 - acc: 0.9750
Epoch 74/200
 - 0s - loss: 0.0606 - acc: 0.9750
Epoch 75/200
 - 0s - loss: 0.0572 - acc: 0.9833
Epoch 76/200
 - 0s - loss: 0.0537 - acc: 0.9750
Epoch 77/200
 - 0s - loss: 0.0514 - acc: 0.9750
Epoch 78/200
 - 0s - loss: 0.0541 - acc: 0.9833
Epoch 79/200
 - 0s - loss: 0.0523 - acc: 0.9833
Epoch 80/200
 - 0s - loss: 0.0562 - acc: 0.9750
Epoch

In [43]:
print(population_desc)

[{'hidden3': (83, 'elu'), 'hidden2': (33, 'softplus'), 'hidden1': (86, 'elu'), 'hidden0': (88, 'softsign'), 'optimizer': 'Adam', 'name': 'Model0', 'learning_rate': 0.0012489600449276383, 'loss_fitness': 3238.4254783613096, 'total_fitness': 3248.4254783613096, 'accuracy_fitness': 10.0}, {'hidden3': (39, 'hard_sigmoid'), 'hidden2': (14, 'selu'), 'hidden1': (94, 'elu'), 'hidden0': (77, 'sigmoid'), 'optimizer': 'Adam', 'name': 'Model1', 'hidden5': (77, 'elu'), 'hidden4': (63, 'elu'), 'learning_rate': 0.0073253442675290095, 'loss_fitness': 28.92639158298877, 'accuracy_fitness': 7.666666507720947, 'total_fitness': 36.59305809070972}, {'hidden3': (67, 'relu'), 'hidden2': (88, 'relu'), 'hidden1': (23, 'linear'), 'hidden0': (65, 'tanh'), 'optimizer': 'Adam', 'name': 'Model2', 'hidden5': (24, 'tanh'), 'hidden4': (23, 'softsign'), 'learning_rate': 0.0015698192360722246, 'loss_fitness': 1476.2712411098234, 'accuracy_fitness': 10.0, 'total_fitness': 1486.2712411098234}, {'hidden3': (87, 'elu'), 'hi

In [40]:
def new_parents():
    parents = []
    for x in population_desc:
        if x.get("total_fitness") >= avg_fitness:
            parents.append((x.get('name'), x.get('total_fitness')))
            difference = x.get('total_fitness') - avg_fitness
            #print(x.get("name")+"    "+str(x.get("total_fitness"))+"    "+str(difference))
    def get_key(item):
        return item[1]
    parents_sorted = sorted(parents, key = get_key, reverse = True)

    if len(parents) % 2 !=0:
        parents_sorted.pop()
        print(parents_sorted)
    else:
        print(parents_sorted)

In [ ]:
def main():
    load_data()
    process_data()
    population()
    grade()
    sort_parents()
    child_populations()
    new_parents()

In [ ]:
if __name__ == "__main__":
    main()